In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
response = requests.get('https://ru.investing.com/crypto')
print(response.status_code)

200


In [3]:
html_content = response.text

soup = BeautifulSoup(html_content, 'lxml')

In [4]:
tbody = soup.find("tbody", class_="datatable-v2_body__8TXQk").find_all("tr")
list_temp = []
for row in tbody:
    temp = row.get_text("|", strip=True).split('|')
    temp[4] = f'{temp[4]}%'
    temp = temp[:3] + temp[6:]
    list_temp.append(temp)
list_btc = list_temp[0]

In [5]:
conn = psycopg2.connect(
    dbname='cripto',
    user='postgres',
    password='postgres',  
    host='postgres', 
    port=5432
)


In [6]:
cur = conn.cursor()

In [7]:
current_date = datetime.now()
current_name = list_btc[0]
current_tag = list_btc[1]
current_price = float(list_btc[2].replace('.', '').replace(',', '.'))
current_dif_1d = float(list_btc[3].replace('.', '').replace(',', '.').replace('%', ''))
current_dif_7d = float(list_btc[4].replace('.', '').replace(',', '.').replace('%', ''))
current_capital = int(float(list_btc[5].replace('$', '').replace(',', '.').replace('T', '')) * 1000000000000)

In [8]:
try:
    insert_sql = '''
    INSERT INTO btc_price (date, name, tag, price, dif_1d, dif_7d, capital) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cur.execute(insert_sql, (current_date, current_name, current_tag, current_price, current_dif_1d, current_dif_7d, current_capital))
    conn.commit()
    print('Запись успешно добавлена!')
except Exception as e:
    print('Ошибка:', e)


Запись успешно добавлена!


In [9]:
#conn.rollback()

In [10]:
cur.close()
conn.close()

In [11]:
list_btc

['Биткоин', 'BTC', '97.425,7', '-0,87%', '-6,28%', '$1,93T']